## Jupyter Notebook for the creation of mdtraj obtained data plots and PCA/tICA analysis


- First you need to import the important libraries. Maybe you need to add more libraries. Feel free to add them in this first cell
- Then you can load the functions that will be used later on. Nice_PES ia a function for plotting FEL plots in a nice way


In [ ]:
# Useful stuff
import numpy as np
from itertools import combinations
import os
import mdtraj as md
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy.ndimage import gaussian_filter
import pandas as pd
import pytraj as pt
import glob

# PyEmma
import pyemma
import pyemma.msm as msm
import pyemma.plots as mpl
import pyemma.coordinates as coor

In [ ]:
def FEL_RES_PYMOL(FEL, file_name):
    a = 0
    b = 1
    color1 = input('Enter a color for PC0:')
    color2 = input('Enter a color for PC1:')
    with open(file_name, 'w') as f:    
        dist_PCA_correl = FEL.feature_PC_correlation
        for ii, icorr in enumerate(dist_PCA_correl.T):
            if ii == a:
                print("For PCA %u"%ii)
                argmaxs = [ii for ii in np.abs(icorr).argsort()[::-1] if not np.isnan(icorr[ii])]
                for art in argmaxs[:10]:
                    print('color ', color1,', resi', (feats[0].describe()[art][9:12]))
                    f.write(pca)
            if ii == b:
                print("For PCA %u"%ii)
                argmaxs = [ii for ii in np.abs(icorr).argsort()[::-1] if not np.isnan(icorr[ii])]
                for art in argmaxs[:10]:
                    print('color ', color2,', resi', (feats[0].describe()[art][9:12]))

In [ ]:
def plot_Nice_PES(P_test,bins=90,sigma=0.99, title=False, size = 1):

    mpl.style.use("seaborn-paper")
    plt.figure(figsize=(6*size,5*size))
    alldata=np.vstack(P_test)
    min1=np.min(alldata[:,0])
    max1=np.max(alldata[:,0])
    min2=np.min(alldata[:,1])
    max2=np.max(alldata[:,1])
    
    tickspacing1=1.0
    tickspacing2=1.0
    z,x,y = np.histogram2d(alldata[:,0], alldata[:,1], bins=bins)
    z += 0.1
    
    # compute free energies
    F = -np.log(z)
    
    
    # contour plot
    extent = [x[0], x[-1], y[0], y[-1]]
    
    plt.xticks(np.arange(int(min1), int(max1)+1, tickspacing1),fontsize=10*size)
    plt.yticks(np.arange(int(min2), int(max2)+1, tickspacing2),fontsize=10*size)
    #    sigma = 0.99 # this depends on how noisy your data is, play with it!
    data = gaussian_filter((F.T)*0.592-np.min(F.T)*0.592, sigma)
    levels=np.linspace(0,np.max(data)-0.5,num=10)
    plt.contour(data,colors='black',linestyles='solid',alpha=0.7,cmap=None, cbar=True, levels=levels,extent=extent)
    plt.contourf(data,alpha=0.5,cmap='jet', cbar=True,levels=levels,extent=extent)
    if title:
        plt.title(title, fontsize = 20*size, y=1.02)
    plt.subplots_adjust(bottom=0.1, right=0.8, top=0.8)
    cax = plt.axes([0.81, 0.1, 0.02, 0.7])
    plt.colorbar(cax=cax, format='%.1f').set_label('Free energy (kcal/mol)', fontsize=10*size, labelpad=5, y= 0.5)
    cax.axes.tick_params(labelsize=10*size)

In [ ]:
import matplotlib as mpl
#import nglview as nv
import ipywidgets as widgets
import pytraj as pt
import warnings
import tqdm
warnings.filterwarnings("ignore")
%pylab inline
%matplotlib inline

def interactive_FEL2(TICA_test, trajfiles, topfile, output_plot=True, n_frames=1,output_structure=False, size=1, bins=90, sigma=0.99, pc1=0, pc2=1):
    '''
        
        Plot the Free Energy Surface(FES) of the given data in a matplotlib pyqt. User can select a point in the FES. 
        
                
        If you experience some problem or you have any comment with this function --> https://github.com/miqueleg/compbiolab :3
        
        Parameters: ---------------------------------------------------------------------------------
        
        data: 3D ndarray (MD,frames,tic_dim). If not, the calculations will be wrong!!
        
        trajfiles: 1D array containing all trajectories strings containing files path and name(order in the list is important)
        
        topfile: string containing file path and name for the .pdb file that will be used as topology.
        
        output_plot: If true, a plot including the FES and selected pint will be displayed at the end.
            default=True
            
        output_structure: If True,the selected structure is also returned as pytraj structure.
            default=False
            
        size: Float/Integer that defines the size of the plots
            default=1
            
        
        Returns: -------------------------------------------------------------------------------------------
        
        - nglview of the structure selected
        
        - If output_structure=True, the selected structure is also returned as pytraj structure
        
        Only in in jupyter-notebook
          
    '''
    ##This imports are kind of necessary in order to plot the graphic in the Tk canvas(PQt5).
    ## At the end of the function, the matplotlib inline canvas has to be setted again in order to do not disturbe next plots
    %matplotlib qt

    ##Plot the Free energy surface as Nice_PES function
    mpl.style.use("seaborn-paper")
    fig, ax = plt.subplots(figsize=(6*size,5*size))
    alldata=np.vstack(TICA_test)
    min1=np.min(alldata[:,pc1])
    max1=np.max(alldata[:,pc1])
    min2=np.min(alldata[:,pc2])
    max2=np.max(alldata[:,pc2])
    print(all_data.shape)

    tickspacing1=1.0
    tickspacing2=1.0
    z,x,y = np.histogram2d(alldata[:,pc1], alldata[:,pc2], bins=bins)
    z += 0.1

    # compute free energies
    F = -np.log(z)

    # contour plot
    extent = [x[0], x[-1], y[0], y[-1]]

    plt.xticks(np.arange(int(min1), int(max1)+1, tickspacing1),fontsize=10*size)
    plt.yticks(np.arange(int(min2), int(max2)+1, tickspacing2),fontsize=10*size)
    #    sigma = 0.99 # this depends on how noisy your data is, play with it!
    data = gaussian_filter((F.T)*0.592-np.min(F.T)*0.592, sigma)
    levels=np.linspace(0,np.max(data)-0.5,num=8)
    ax.contour(data,colors='black',linestyles='solid',alpha=0.7,cmap=None, cbar=True, levels=levels,extent=extent)
    ax.contourf(data,alpha=0.5,cmap='jet', cbar=True,levels=levels,extent=extent)

    ##ginput will wait until a click is done into the plot and collect the coordinates into the x variable
    x = plt.ginput()


    #nodes are the prepared data corresponding of the points on the TICA scatter (tic0 and tic1 by default)
    nodes = np.array(list(zip(alldata[:,pc1],alldata[:,pc2])))
    node = np.array(x[0])
    #cdist finction computes the distance matrix between TICA nodes and node
    distances = np.array(cdist([node], nodes))
    #clicked frame is the position in the matrix distance in wich you can find the minimun value
    clicked_frames = np.argsort(distances)[0]
    clicked_frames = clicked_frames[:n_frames]
    MD_out = pt.Trajectory(top=topfile)
    
    for f in tqdm.tqdm(range(len(clicked_frames))):
        clicked_frame = clicked_frames[f]
        MD=pt.iterload(trajfiles, top=topfile)
        MD_out.append(MD[clicked_frame])

        #Frames is a list containing the len of all the MD's.
        #This forces the input TICA_test to have a 3D shape (MD,frames,tic_dim). If not, the calculations are wrong!!
        Frames = []
        for traj in TICA_test:
            Frames.append(len(traj))
        #In this loop is computed in wich MD and MD_frame you can find the clicked frame
        MD = 0
        Frame = clicked_frame
        for i in list(range(len(Frames))):
            sum_frames = sum([Frames[j] for j in list(range(len(Frames))) if j <= i])
            if clicked_frame <= sum_frames:
                MD = i
                Frame = clicked_frame - sum([Frames[j] for j in list(range(len(Frames))) if j < i])
                break
            else:
                continue
    #The selected_frame is displayed using pytraj and nglview
    MD=pt.iterload(trajfiles, top=topfile)
    view=nv.show_pytraj(MD)
    view.clear()
    view.add_cartoon(selection="protein")
    view.frame = int(clicked_frame)

    #returning to the matplotlib inline
    %matplotlib inline
    #plot the selected position with a scatter
    if output_plot:
        plt.figure(figsize=(6*size,5*size))
        plt.contour(data,colors='black',linestyles='solid',alpha=0.7,cmap=None, cbar=True, levels=levels,extent=extent)
        plt.contourf(data,alpha=0.2,cmap='jet', cbar=True,levels=levels,extent=extent)
        plt.scatter(np.array(x)[:,0],np.array(x)[:,1], c ='b', s =50)
    if output_structure:
        return view, MD_out
    else:
        return view


# 1 - Discretization

I start by defining the input files names and the input coordinates

In [ ]:
#### APO variants ####

directo= 
trajfiles_HG3_apo=[fi for fi in glob.glob(directo + '*_super.dcd')]
topfile_HG3_apo=

directo=
trajfiles_HG317_apo=[fi for fi in glob.glob(directo + '*_super.dcd')]
topfile_HG317_apo=

directo=
trajfiles_HG317_shell_apo=[fi for fi in glob.glob(directo + '*_super.dcd')]
topfile_HG317_shell_apo=

directo=
trajfiles_HG4_apo=[fi for fi in glob.glob(directo + '*_super.dcd')]
topfile_HG4_apo=


##### TS variants ####

directo=
trajfiles_HG3_ts=[fi for fi in glob.glob(directo + '*_super.dcd')]
topfile_HG3_ts=

directo=
trajfiles_HG317_ts=[fi for fi in glob.glob(directo + '*_super.dcd')]
topfile_HG317_ts=

directo=
trajfiles_HG317shell_ts=[fi for fi in glob.glob(directo + '*_super.dcd')]
topfile_HG317shell_ts=

directo=
trajfiles_HG4_ts=[fi for fi in glob.glob(directo + '*_super.dcd')]
topfile_HG4_ts=


In [ ]:
trajfiles_GCV_ATP


In [ ]:
trajfiles = []
trajfiles.append(trajfiles_HG3_apo)
trajfiles.append(trajfiles_HG317_apo)
trajfiles.append(trajfiles_HG317shell_apo)
trajfiles.append(trajfiles_HG4_apo)
trajfiles.append(trajfiles_HG3_ts)
trajfiles.append(trajfiles_HG317_ts)
trajfiles.append(trajfiles_HG317shell_ts)
trajfiles.append(trajfiles_HG4_ts)


topfiles = []
topfiles.append(topfile_HG3_apo)
topfiles.append(topfile_HG317_apo)
topfiles.append(topfile_HG317shell_apo)
topfiles.append(topfile_HG4_apo)
topfiles.append(topfile_HG3_ts)
topfiles.append(topfile_HG317_ts)
topfiles.append(topfile_HG317shell_ts)
topfiles.append(topfile_HG4_ts)

In [ ]:
trajfiles

In [ ]:
topfiles

## Analysis:

We normaly organize the data as the following: 
- FEATS are the lists variables that contains the data that will be used to extract info from the MD's (only based in the topfile). Feats list contain the feats for all systems. ej: feat1 == feats[0]
- SOURCES are the arrays containing the values of the feats for all frames

# PCA / TICA

In [ ]:
feat1=pyemma.coordinates.featurizer(topfiles[0])
all_Ca1=feat1.select_Ca() # Features selected CA coordinates. The trajectories MUST be aligned first!
print(all_Ca1)            # if you want to avoid the aligning, you can use: feat1.add_distances_ca()
feat1.add_selection(all_Ca1)

feat2=pyemma.coordinates.featurizer(topfiles[1])
all_Ca2=feat2.select_Ca()
print(all_Ca2)
feat2.add_selection(all_Ca2)

feat3=pyemma.coordinates.featurizer(topfiles[2])
all_Ca3=feat3.select_Ca()
print(all_Ca3)
feat3.add_selection(all_Ca3)

feat4=pyemma.coordinates.featurizer(topfiles[3])
all_Ca4=feat4.select_Ca()
print(all_Ca4)
feat4.add_selection(all_Ca4)

feat5=pyemma.coordinates.featurizer(topfiles[4])
all_Ca5=feat5.select_Ca()
print(all_Ca5)
feat5.add_selection(all_Ca5)

feat6=pyemma.coordinates.featurizer(topfiles[5])
all_Ca6=feat6.select_Ca()
print(all_Ca6)
feat6.add_selection(all_Ca6)

feat7=pyemma.coordinates.featurizer(topfiles[6])
all_Ca7=feat7.select_Ca()
print(all_Ca7)
feat7.add_selection(all_Ca7)

feat8=pyemma.coordinates.featurizer(topfiles[7])
all_Ca8=feat8.select_Ca()
print(all_Ca8)
feat8.add_selection(all_Ca8)


In [ ]:
print(len(feat1.describe()))
print(len(feat2.describe()))
print(len(feat3.describe()))
print(len(feat4.describe()))
print(len(feat5.describe()))
print(len(feat6.describe()))
print(len(feat7.describe()))
print(len(feat8.describe()))


In [ ]:
feats=[]
feats.append(feat1)
feats.append(feat2)
feats.append(feat3)
feats.append(feat4)
feats.append(feat5)
feats.append(feat6)
feats.append(feat7)
feats.append(feat8)


In [ ]:
# This may take a while

sources=[]
print (len(feats[0].describe()))
for traj in range(0,len(topfiles)):
    print (traj,len(trajfiles))
    sources.append(pyemma.coordinates.load(trajfiles[traj],features=feats[traj]))

In [ ]:
np.array(sources).shape

In [ ]:
joint_data = sources[0] + sources[1] + sources[2] + sources[3] + sources[4] + sources[5] + sources[6] + sources[7]

## PCA

- Here the PCA space is created using all the data from all the systems. This can be done if we have the same features for all the different systems.
- Later on, the data is splitted for the different systems, but the space is the same

In [ ]:
pca_joint=pyemma.coordinates.pca(data=joint_data,kinetic_map=False,stride=1)
pca_Y = pca_joint.get_output()
pca_HG3_apo,pca_HG317_apo, pca_HG317shell_apo,pca_HG4_apo, pca_HG3_ts,pca_HG317_ts, pca_HG317shell_ts,pca_HG4_ts, = np.split(pca_Y, np.cumsum(list(map(len, trajfiles)))[:-1])

In [ ]:
all_data=np.vstack(pca_Y)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
extend=axis()

In [ ]:
all_data=np.vstack(pca_HG3_apo)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)

all_data=np.vstack(pca_HG317_apo)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)

all_data=np.vstack(pca_HG317shell_apo)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)

all_data=np.vstack(pca_HG_4apo)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)

all_data=np.vstack(pca_HG3_ts)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)

all_data=np.vstack(pca_HG317_ts)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)

all_data=np.vstack(pca_HG317shell_ts)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)

all_data=np.vstack(pca_HG4_ts)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)


Here you will see the contributions of each feat (or the most important feats) for each PCA dimension (PC)

In [ ]:
dist_PCA_correl = pca_joint.feature_PC_correlation

n_argmax = 2
for ii, icorr in enumerate(dist_PCA_correl.T):
    print("For PCA %u"%ii)
    argmaxs = [ii for ii in np.abs(icorr).argsort()[::-1] if not np.isnan(icorr[ii])]
    for art in argmaxs[:15]:
        print(feats[0].describe()[art][9:12])
    print()

In [ ]:
import pickle

pickle.dump(pca_HG3_apo, open ("FELS_JOINT/pca_HG3_apo.p", "wb"))
pickle.dump(pca_HG317_apo, open ("FELS_JOINT/pca_HG317_apo.p", "wb"))
pickle.dump(pca_HG317shell_apo, open ("FELS_JOINT/pca_HG317shell_apo.p", "wb"))
pickle.dump(pca_HG4_apo, open ("FELS_JOINT/pca_HG4_apo.p", "wb"))
pickle.dump(pca_HG3_ts, open ("FELS_JOINT/pca_HG3_ts.p", "wb"))
pickle.dump(pca_HG317_ts, open ("FELS_JOINT/pca_HG317_ts.p", "wb"))
pickle.dump(pca_HG317shell_ts, open ("FELS_JOINT/pca_HG317shell_ts.p", "wb"))
pickle.dump(pca_HG4_ts, open ("FELS_JOINT/pca_HG4_ts.p", "wb"))

## tICA

The biggest difference between PCA and tICA is the introduction of the lag time parameter. With this parameter we can ignore fast movements that are not important for the system like vibrations and side chain rotations. Is important to chose a propper lag time that reduces the number of dimensions that explains the variations in the MD's.

To select a initial lag time, we use this code to sample different values and we select the one that explains the same variation in less dimensions (minimum in the plot)

###### This part may take a while

In [ ]:
#Compute TICA on the combined data from all trajectories

multi_tica=[]

for i in [10,50,100,200,500,750,1000]:
    tica_joint=pyemma.coordinates.tica(data=[t for t in joint_data if len(t)>1500],lag=i,stride=1)
    multi_tica.append([i,[tica_joint.lag, tica_joint.ndim]])
    del(tica_joint)
    
plt.plot([t[1][0] for t in multi_tica], [t[1][1]for t in multi_tica])
plt.xlabel('lagtime')
plt.ylabel('dimensions')
plt.show()
for tica in multi_tica:
    print(tica) 

## TICA

In [ ]:
#%%time
lag=200   # We add the lag time that we selected in the previous part
tica_joint=pyemma.coordinates.tica(data=joint_data,lag=lag,kinetic_map=False,stride=1)
#kmeans=pyemma.coordinates.cluster_kmeans(data=None,k=25,max_iter=100)
#disc = pyemma.coordinates.discretizer(joint_data,transform=tica_joint,cluster=kmeans,stride=1)

#We get the output from tica on the combined dataset
Y_test = tica_joint.get_output()

#We know divide the tica output to recover the tica object for each dataset, i.e. tica for WT, tica for Lovd1..

tica_APO,tica_APOMG, tica_THM, tica_GCV, tica_THM_ATP, tica_GCV_ATP, tica_TMP_ADP, tica_GMP_ADP = np.split(Y_test, np.cumsum(list(map(len, trajfiles)))[:-1])

In [ ]:
#We now compute the free energy of the combined dataset and separated datasets

dims = 0,1

all_data=np.vstack(Y_test)
pyemma.plots.plot_free_energy(all_data[:,dims[0]], all_data[:,dims[1]], avoid_zero_count=1)
# extend=axis()

all_data=np.vstack(tica_HG3_apo)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)

all_data=np.vstack(tica_HG317_apo)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)

all_data=np.vstack(tica_HG317shell_apo)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)

all_data=np.vstack(tica_HG_4apo)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)

all_data=np.vstack(tica_HG3_ts)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)

all_data=np.vstack(tica_HG317_ts)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)

all_data=np.vstack(tica_HG317shell_ts)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)

all_data=np.vstack(tica_HG4_ts)
pyemma.plots.plot_free_energy(all_data[:,0], all_data[:,1], avoid_zero_count=1)
axis(extend)

In [ ]:
dist_TICA_correl = tica_joint.feature_TIC_correlation

n_argmax = 2
for ii, icorr in enumerate(dist_TICA_correl.T):
    print("For TICA %u"%ii)
    argmaxs = [ii for ii in np.abs(icorr).argsort()[::-1] if not np.isnan(icorr[ii])]
    for art in argmaxs[:15]:
        print(feats[0].describe()[art][9:12])
    print()

In [ ]:
pickle.dump(pca_HG3_apo, open ("FELS_JOINT/pca_HG3_apo.p", "wb"))
pickle.dump(pca_HG317_apo, open ("FELS_JOINT/pca_HG317_apo.p", "wb"))
pickle.dump(pca_HG317shell_apo, open ("FELS_JOINT/pca_HG317shell_apo.p", "wb"))
pickle.dump(pca_HG4_apo, open ("FELS_JOINT/pca_HG4_apo.p", "wb"))
pickle.dump(pca_HG3_ts, open ("FELS_JOINT/pca_HG3_ts.p", "wb"))
pickle.dump(pca_HG317_ts, open ("FELS_JOINT/pca_HG317_ts.p", "wb"))
pickle.dump(pca_HG317shell_ts, open ("FELS_JOINT/pca_HG317shell_ts.p", "wb"))
pickle.dump(pca_HG4_ts, open ("FELS_JOINT/pca_HG4_ts.p", "wb"))